# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [62]:
#!pip install -qU ragas==0.2.10

In [63]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [1]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [2]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [4]:
!mkdir data

In [5]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31287    0 31287    0     0  92995      0 --:--:-- --:--:-- --:--:-- 92839


In [6]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0   182k      0 --:--:-- --:--:-- --:--:--  183k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [4]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [5]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
#ENM added the following 4 lines - all lines added to deal with API Retry Rate Issues
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from tenacity import retry, wait_random_exponential, stop_after_attempt
set_llm_cache(InMemoryCache())


#generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))


generator_llm = LangchainLLMWrapper(
    ChatOpenAI(
       # model="gpt-4",
        # model="gpt-3.5-turbo",
        model="gpt-4o", #ENM ADDED THE REMAINING FOR Rate issues
        temperature=0,
        request_timeout=30,
        max_retries=5
        #time.sleep(2) 
        #retry_decorator=retry(
        #    wait=wait_random_exponential(min=1, max=60),
        #    stop=stop_after_attempt(3)
        #)
        #model_kwargs={"retry_delay":60}
    )
)

#@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
#def completion_with_backoff(**kwargs):
#    return openai.Completion.create(**kwargs)
# completion_with_backoff(model="text-davinci-002", prompt="Once upon a time,")

generator_embeddings = LangchainEmbeddingsWrapper(
    OpenAIEmbeddings(#ENM ADDED THE REMAINING FOR Rate issues
        max_retries=5
        #time.sleep(2) 
        #retry_decorator=retry(
        #    wait=wait_random_exponential(min=1, max=60),
        #    stop=stop_after_attempt(3)
        #)
    #model_kwargs={"retry_delay":60}
    )
)

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [6]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [7]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

ENM NOTE 1:   SO FAR, 3/2/25 - NONE OF THE RATE LIMITERS WORKED - JUST CHANGED GPT MODEL TO 3.5-TURBO
Ultimately - this didn't work either - needed to ask OpenAI why my access was stuck at free instead of TIER 1 and they claimed a "bug" which was covered in many Reddit posts. Once I was upgraded, the problems disappeared

USE THE FOLLOWING TO TRY AND DEAL WITH THE RATE LIMIT ISSUES WITH OPENAI

STILL RAN INTO ERRORS

In [ ]:
import time
import backoff
from ragas.testset.transforms import default_transforms, apply_transforms
from openai import OpenAIError  

# Function to retry on rate limit errors using exponential backoff
@backoff.on_exception(backoff.expo, OpenAIError , max_tries=7, max_time=120)
def apply_transforms_with_retry(kg, transforms):
    """Applies transformations with a retry mechanism on rate-limit errors."""
    return apply_transforms(kg, transforms)

# Ensure a delay before applying transformations (optional)
time.sleep(20)

# Define Transformer and Embedding Model
transformer_llm = generator_llm
embedding_model = generator_embeddings  # Ensure this is initialized correctly

# Apply transformations with retry
try:
    default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
    kg = apply_transforms_with_retry(kg, default_transforms)
    print("Transformations applied successfully!")
except OpenAIError as e:
    print(f"Failed due to OpenAI API error: {e}")

kg  # Return or print the knowledge graph

ENM NOTE 2:

TRYING THIS MORE DIRECT APPROACH TO EDITING BATCH SIZES AND OTHER THINGS - ALSO DID NOT WORK

In [ ]:
import time
import backoff
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from ragas.testset.transforms import default_transforms, apply_transforms
from openai import OpenAIError  # Correct import for OpenAI API errors
from langchain_core.documents.base import Document

# Use GPT-3.5-turbo for lower token costs (optional)
#MODEL_NAME = "gpt-3.5-turbo"  # Change back to "gpt-4o" if needed
MODEL_NAME = "gpt-4o"#gpt-3.5-turbo"  # Change back to "gpt-4o" if needed

# Initialize the LLM Wrapper
generator_llm = LangchainLLMWrapper(
    ChatOpenAI(
        model=MODEL_NAME,
        temperature=0,
        request_timeout=30,
        max_retries=5  # OpenAI's built-in retry mechanism
    )
)

# Function to retry API calls when hitting rate limits
@backoff.on_exception(backoff.expo, OpenAIError, max_tries=7, max_time=120)
def apply_transforms_with_retry(kg, transforms):
    """Applies transformations with a retry mechanism on rate-limit errors."""
    return apply_transforms(kg, transforms)

# **Step 1: Reduce Input Size Before Making API Calls**
#def truncate_documents(docs, max_tokens=2500):
#    """Truncate documents to fit within token limits."""
#    truncated_docs = []
#    for doc in docs:
#        if len(doc) > max_tokens:
#            truncated_docs.append(doc[:max_tokens])  # Keep first max_tokens characters
#        else:
#            truncated_docs.append(doc)
#    return truncated_docs
#
#docs = truncate_documents(docs)  # Apply truncation to reduce size

# Step 1: Truncate Documents
truncated_texts = truncate_documents(docs)  # Get list of truncated strings

# Step 2: Convert Truncated Strings Back into LangChain Document Objects
truncated_docs = [Document(page_content=text) for text in truncated_texts]

# Step 3: Apply Transforms with Correct Document Format
default_transforms = default_transforms(documents=truncated_docs, llm=transformer_llm, embedding_model=embedding_model)

BATCH_SIZE = 10  # Adjust based on API limits
for i in range(0, len(default_transforms), BATCH_SIZE):
    batch = default_transforms[i : i + BATCH_SIZE]
    try:
        kg = apply_transforms_with_retry(kg, batch)
        time.sleep(60)  # Introduce delay between batch requests
        print(f"Processed batch {i//BATCH_SIZE + 1}/{(len(default_transforms) // BATCH_SIZE) + 1}")
    except OpenAIError as e:
        print(f"Failed due to OpenAI API error: {e}")

kg  # Return or print the knowledge graph


ENM Note3:  Another trial, many of these trials were attempts with Todd that just weren't working.  I would love to figure out a way to make them work even if it took forever

In [ ]:
import time

# Define a function that adds a delay to a Completion API call
def delayed_completion(delay_in_seconds: float = 1, **kwargs):
    """Delay a completion by a specified amount of time."""

    # Sleep for the delay
    time.sleep(delay_in_seconds)

    # Call the Completion API and return the result
    return client.chat.completions.create(**kwargs)


# Calculate the delay based on your rate limit
rate_limit_per_minute = 3  # ENM'S CURRENT OPENAI RATE LIMIT IS 3PM
# ENM-USING 55 sec TO ENSURE BELOW THE MINUTE RATE LIMIT
delay = 55 / rate_limit_per_minute

delayed_completion(
    delay_in_seconds=delay,
    #ALSO CONSIDER USING MODELS WITH HIGHER LIMITS
    #model="gpt-3.5-turbo",
    model="gpt-4o", 
    messages=[{"role": "user", "content": "Once upon a time,"}]
)

# THE FOLLOWING WAS SOME OTHER CODE THAT I TRIED
'''
import time
import backoff
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from ragas.testset.transforms import default_transforms, apply_transforms
from openai import OpenAIError  # Correct import for OpenAI API errors
from langchain_core.documents.base import Document

# Use GPT-3.5-turbo for lower token costs (optional)
#MODEL_NAME = "gpt-3.5-turbo"  # Change back to "gpt-4o" if needed
MODEL_NAME = "gpt-4o"#gpt-3.5-turbo"  # Change back to "gpt-4o" if needed

# Initialize the LLM Wrapper
generator_llm = LangchainLLMWrapper(
    ChatOpenAI(
        model=MODEL_NAME,
        temperature=0,
        request_timeout=30,
        max_retries=5  # OpenAI's built-in retry mechanism
    )
)

# Function to retry API calls when hitting rate limits
@backoff.on_exception(backoff.expo, OpenAIError, max_tries=7, max_time=120)
def apply_transforms_with_retry(kg, transforms):
    """Applies transformations with a retry mechanism on rate-limit errors."""
    return apply_transforms(kg, transforms)

# **Step 1: Reduce Input Size Before Making API Calls**
#def truncate_documents(docs, max_tokens=2500):
#    """Truncate documents to fit within token limits."""
#    truncated_docs = []
#    for doc in docs:
#        if len(doc) > max_tokens:
#            truncated_docs.append(doc[:max_tokens])  # Keep first max_tokens characters
#        else:
#            truncated_docs.append(doc)
#    return truncated_docs
#
#docs = truncate_documents(docs)  # Apply truncation to reduce size

# Step 1: Truncate Documents
truncated_texts = truncate_documents(docs)  # Get list of truncated strings

# Step 2: Convert Truncated Strings Back into LangChain Document Objects
truncated_docs = [Document(page_content=text) for text in truncated_texts]

# Step 3: Apply Transforms with Correct Document Format
default_transforms = default_transforms(documents=truncated_docs, llm=transformer_llm, embedding_model=embedding_model)

BATCH_SIZE = 10  # Adjust based on API limits
for i in range(0, len(default_transforms), BATCH_SIZE):
    batch = default_transforms[i : i + BATCH_SIZE]
    try:
        kg = apply_transforms_with_retry(kg, batch)
        time.sleep(60)  # Introduce delay between batch requests
        print(f"Processed batch {i//BATCH_SIZE + 1}/{(len(default_transforms) // BATCH_SIZE) + 1}")
    except OpenAIError as e:
        print(f"Failed due to OpenAI API error: {e}")

kg  # Return or print the knowledge graph

'''

ENM TRIAL 4: ALSO DIDN'T WORK

In [ ]:
import time
import backoff
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from ragas.testset.transforms import default_transforms, apply_transforms
from openai import OpenAIError

# Initialize the LLM Wrapper
generator_llm = LangchainLLMWrapper(
    ChatOpenAI(
        model="gpt-4o",  # Consider using "gpt-3.5-turbo" for higher RPM limits
        temperature=0,
        request_timeout=30,
        max_retries=5
    )
)

# OpenAI rate limits
MAX_REQUESTS_PER_MINUTE = 3  # Update this based on your OpenAI account limits
DELAY_BETWEEN_REQUESTS = 65 / MAX_REQUESTS_PER_MINUTE  # Calculate safe delay

# Function to retry API calls when hitting rate limits
@backoff.on_exception(backoff.expo, OpenAIError, max_tries=5, max_time=300)
def apply_transforms_with_retry(kg, batch):
    """Applies transformations with a retry mechanism on rate-limit errors."""
    return apply_transforms(kg, batch)

# Convert truncated text back into LangChain Document format
from langchain_core.documents.base import Document
truncated_texts = truncate_documents(docs)
truncated_docs = [Document(page_content=text) for text in truncated_texts]

# Apply transforms in batches with proper delays
BATCH_SIZE = 2  # Adjust based on API limits
for i in range(0, len(truncated_docs), BATCH_SIZE):
    batch = truncated_docs[i : i + BATCH_SIZE]
    
    try:
        kg = apply_transforms_with_retry(kg, batch)
        print(f"Processed batch {i//BATCH_SIZE + 1}/{(len(truncated_docs) // BATCH_SIZE) + 1}")
        
        # **Smart Delay to Avoid Rate Limits**
        time.sleep(DELAY_BETWEEN_REQUESTS)

    except OpenAIError as e:
        print(f"Failed due to OpenAI API error: {e}")

kg  # Return or print the knowledge graph


ENM TRIAL 5 - ALSO DIDN'T WORK

In [ ]:
import time
import backoff
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from ragas.testset.transforms import default_transforms, apply_transforms
from openai import OpenAIError
from langchain_core.documents.base import Document

# Initialize the LLM Wrapper
generator_llm = LangchainLLMWrapper(
    ChatOpenAI(
        model="gpt-4o",  # Consider using "gpt-3.5-turbo" for higher RPM limits
        temperature=0,
        request_timeout=30,
        max_retries=3
    )
)

# OpenAI rate limits
MAX_REQUESTS_PER_MINUTE = 3  
DELAY_BETWEEN_REQUESTS = 65 / MAX_REQUESTS_PER_MINUTE  

# Function to retry API calls when hitting rate limits
@backoff.on_exception(backoff.expo, OpenAIError, max_tries=5, max_time=300)
def apply_transforms_with_retry(kg, batch):
    """Applies transformations with a retry mechanism on rate-limit errors."""
    return apply_transforms(kg, batch)

# Convert LangChain Document objects into a format that `apply_transforms()` expects
formatted_docs = [{"text": doc.page_content} for doc in truncated_docs]  

# Apply default transforms correctly
default_transforms = default_transforms(documents=formatted_docs, llm=generator_llm, embedding_model=embedding_model)

# Apply transformations in batches with a delay to prevent rate limits
BATCH_SIZE = 2  
for i in range(0, len(default_transforms), BATCH_SIZE):
    batch = default_transforms[i : i + BATCH_SIZE]
    
    try:
        kg = apply_transforms_with_retry(kg, batch)
        print(f"Processed batch {i//BATCH_SIZE + 1}/{(len(default_transforms) // BATCH_SIZE) + 1}")

        # Smart delay to avoid rate limits
        time.sleep(DELAY_BETWEEN_REQUESTS)

    except OpenAIError as e:
        print(f"Failed due to OpenAI API error: {e}")

kg  # Return or print the knowledge graph

ENM NOTE ON DOCS - RECEIVED AN ERROR FROM CURSOR GPT THAT "maybe" THE ISSUE WAS THE DOCUMENT FOR FACTOR SO PRINTED IT OUT TO SEE THAT IT WAS OK

In [102]:
print(type(batch))  # Should be a list of documents or processed chunks
print(batch[:2])  # Show the first 2 elements

<class 'list'>
[Document(metadata={}, page_content='Simon Willison’s Weblog\n\nSubscribe\n\nThings we learned about LLMs in 2024\n\n31st December 2024\n\nA lot has happened in the world of Large Language Models over the course of 2024. Here’s a review of things we figured out about the field in the past twelve months, plus my attempt at identifying key themes and pivotal moments.\n\nThis is a sequel to my review of 2023.\n\nIn this article:\n\nThe GPT-4 barrier was comprehensively broken\n\nSome of those GPT-4 models run on my laptop\n\nLLM prices crashed, thanks to competition and increased efficiency\n\nMultimodal vision is common, audio and video are starting to emerge\n\nVoice and live camera mode are science fiction come to life\n\nPrompt driven app generation is a commodity already\n\nUniversal access to the best models lasted for just a few short months\n\n“Agents” still haven’t really happened yet\n\nEvals really matter\n\nApple Intelligence is bad, Apple’s MLX library is excel

ENM NOTE 6: THIS WAS POSTED BY KAT ON DISCORD BUT UNSURE IF IT WAS FOR THIS ASSIGNMENT OR THE NEXT ONE(S)

In [ ]:
# Kat posted something in the Discord Discussions that had the following:
from langchain.chat_models import init_chat_model 
from langchain_core.rate_limiters import InMemoryRateLimiter 
rate_limiter = InMemoryRateLimiter( 
	requests_per_second=1, # <-- make a request once every 1 seconds!! 
	check_every_n_seconds=0.1, # Wake up every 100 ms to check whether allowed to make a request, max_bucket_size=10, # Controls the maximum burst size.
	 )
 model = init_chat_model("gpt-4o", temperature=0, rate_limiter=rate_limiter) 
 generator_llm = LangchainLLMWrapper(model) 
 generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings()) 

from ragas.testset import TestsetGenerator 
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings) 
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

# From <https://discord.com/channels/1135695983720792216/1327051742638116894/threads/1342710885277237268> 

ENM NOTE 7:  THIS WAS A TRIAL AS WELL BUT DIDN'T NEED TO USE IT

In [ ]:
# THIS WAS THE ORIGINAL CODE slightly modified THAT RAN INTO RATE LIMIT ERRORS LEADING TO THE CREATION OF THE CODES ABOVE

from ragas.testset.transforms import default_transforms, apply_transforms
from langchain.chat_models import init_chat_model 
from langchain_core.rate_limiters import InMemoryRateLimiter 
rate_limiter = InMemoryRateLimiter( 
	requests_per_second=0.045, # <-- make a request 2.7X/MIN!! 
	check_every_n_seconds=0.1, # Wake up every 100 ms to check whether allowed to make a request, max_bucket_size=10, # Controls the maximum burst size.
	 )

model = init_chat_model("gpt-4o", temperature=0, rate_limiter=rate_limiter) 
generator_llm = LangchainLLMWrapper(model) 
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings()) 

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

'''
# ALL OF THE FOLLOWING IS FROM KAT'S CODE - NEED TO ADJUST TO WORK WITH THE CODE ABOVE...
from langchain.chat_models import init_chat_model 
from langchain_core.rate_limiters import InMemoryRateLimiter 
rate_limiter = InMemoryRateLimiter( 
	requests_per_second=1, # <-- make a request once every 1 seconds!! 
	check_every_n_seconds=0.1, # Wake up every 100 ms to check whether allowed to make a request, max_bucket_size=10, # Controls the maximum burst size.
	 )
 model = init_chat_model("gpt-4o", temperature=0, rate_limiter=rate_limiter) 
 generator_llm = LangchainLLMWrapper(model) 
 generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings()) 

from ragas.testset import TestsetGenerator 
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings) 
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)
'''

ENM NOTE 8: THE FOLLOWING WAS THE ORIGINAL CODE WITH NO DELAYS

SKIP THE FOLLOWING IF USING ANY OF THE FORMER NOTE/TRIALS;
THIS ULTIMATELY WORKED AFTER OPENAI GOT ME TIER 1 ACCESS .... *sigh* BUT LOST 2 WEEKS TRYING TO GET AROUND RATELIMIT ERRORS

In [8]:
# ORIGINAL NOTEBOOK CODE
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 70)

We can save and load our knowledge graphs as follows.

In [9]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 70)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [10]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

In [ ]:
# Kat posted something in the Discord Discussions that had the following for Rate Limit issues with TestGenerator:
# NOT SURE IF APPLICABLE OR NOT - NOT EXACTLY THE SAME, NEED TO STUDY BEFORE USING - COULD BE A DIFFERENT HW ASSIGNMENT
from langchain.chat_models import init_chat_model 
from langchain_core.rate_limiters import InMemoryRateLimiter 
rate_limiter = InMemoryRateLimiter( 
	requests_per_second=1, # <-- make a request once every 1 seconds!! 
	check_every_n_seconds=0.1, # Wake up every 100 ms to check whether allowed to make a request, max_bucket_size=10, # Controls the maximum burst size.
	 )
 model = init_chat_model("gpt-4o", temperature=0, rate_limiter=rate_limiter) 
 generator_llm = LangchainLLMWrapper(model) 
 generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings()) 

from ragas.testset import TestsetGenerator 
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings) 
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

# From <https://discord.com/channels/1135695983720792216/1327051742638116894/threads/1342710885277237268> 

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [16]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.
(SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5), - Weighting of 50% of the queries to utilize single-hop (one-step retrievals) and is used to help assess  documents given a straightforward, non-multi-step questions, from the source material.
(MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25), - weighted 25% - creates questons that require synthesizing information from multiple parts of the source material and forms more abstract conclusions. 
(MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25), weighted 25% - generates questions that need multiple pieces of specific information to be combined for the answer.  
The mix of query types helps test both basic retrieval and more complex.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [17]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Why Apple Intelligence bad but MLX library good?,[Prompt driven app generation is a commodity a...,"Apple Intelligence is considered bad, whereas ...",single_hop_specifc_query_synthesizer
1,What is the cost of using Anthropic's Claude 3...,"[gets you OpenAI’s most expensive model, o1. G...",Anthropic’s Claude 3 Haiku model costs $0.25 p...,single_hop_specifc_query_synthesizer
2,What is the significance of the new WebRTC API...,[your camera feed with the model and talk abou...,The new WebRTC API introduced by OpenAI is sig...,single_hop_specifc_query_synthesizer
3,How does Vercel's approach to prompt protectio...,[“agents” as dependent on AGI itself. A model ...,"Vercel's Malte Ubl mentioned that initially, t...",single_hop_specifc_query_synthesizer
4,new york flights cost how much?,[day after that. DeepSeek v3 is a huge 685B pa...,The largest Llama 3 model cost about the same ...,single_hop_specifc_query_synthesizer
5,How has the concept of prompt-driven app gener...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, prompt-driven app generation has beco...",multi_hop_abstract_query_synthesizer
6,How has the environmental impact of AI evolved...,[<1-hop>\n\nPrompt driven app generation is a ...,The environmental impact of AI has improved du...,multi_hop_abstract_query_synthesizer
7,How has the concept of prompt-driven app gener...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, prompt-driven app generation has beco...",multi_hop_abstract_query_synthesizer
8,What were the significant developments in larg...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2024, significant developments in large lan...",multi_hop_specific_query_synthesizer
9,What were the key developments in Large Langua...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2023, Large Language Models (LLMs) were rec...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [18]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [19]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What advancements have been made with Claude 3...,[Prompt driven app generation is a commodity a...,"Anthropic's Claude 3 series launched in March,...",single_hop_specifc_query_synthesizer
1,Wht are the key features and pricing details o...,"[gets you OpenAI’s most expensive model, o1. G...","Anthropic’s Claude 3 Haiku, released in March,...",single_hop_specifc_query_synthesizer
2,What are the key features and impacts of Claud...,[your camera feed with the model and talk abou...,Claude 3.5 Sonnet introduced groundbreaking fe...,single_hop_specifc_query_synthesizer
3,How does Apple's MLX library enhance the perfo...,[“agents” as dependent on AGI itself. A model ...,"Apple's MLX library, described as an array fra...",single_hop_specifc_query_synthesizer
4,How have advancements in GPT-4 and the competi...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in GPT-4 and the subsequent compe...,multi_hop_abstract_query_synthesizer
5,How has the evolution of prompt-driven app gen...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, prompt-driven app generation has beco...",multi_hop_abstract_query_synthesizer
6,How have advancements in GPT-4 and the competi...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in GPT-4 and the subsequent compe...,multi_hop_abstract_query_synthesizer
7,How did the shift from universal access to sub...,[<1-hop>\n\nPrompt driven app generation is a ...,The shift from universal access to subscriptio...,multi_hop_abstract_query_synthesizer
8,How has Google's Gemini series contributed to ...,[<1-hop>\n\nPrompt driven app generation is a ...,Google's Gemini series has significantly contr...,multi_hop_specific_query_synthesizer
9,How have the advancements in GPT-4o and its pr...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"The advancements in GPT-4o, particularly its p...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [20]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [21]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [22]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [24]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [25]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [26]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [27]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [29]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [30]:
rag_chain.invoke({"question" : "What are Agents?"})

'Agents refer to AI systems that can act on behalf of users, but the term is considered vague and lacks a clear, widely understood definition. There are different interpretations of what agents are, including those that perform tasks like a travel agent and those that utilize large language models (LLMs) with access to tools for problem-solving. Overall, there is skepticism about their utility, largely due to challenges in determining truth from fiction, which is critical for effective decision-making.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [31]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [32]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'frosty-availability-23' at:
https://smith.langchain.com/o/2928bae9-0c08-4543-b4d4-5b2f0a267e9d/datasets/00704a1a-6702-4a89-8b26-920db81b70ae/compare?selectedSessions=f1e25c62-0089-4995-88f3-d697d73f79f9




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How did the developments in GPT-4 and Claude A...,The developments in GPT-4 and Claude 3.5 Sonne...,None,"In 2023, GPT-4 and Claude Artifacts significan...",0,0,0,0.506365,c59f1bae-1f08-4850-a9ef-0c70a42536f4,8c56a0c7-8514-4b54-a6cd-2b465fccce2d
1,How did the developments in Large Language Mod...,"In 2023, the developments in Large Language Mo...",None,"In 2023, Large Language Models (LLMs) marked a...",1,1,0,0.223102,e2093edc-a5c9-4077-ae52-81962af78e16,fe03f3b9-2404-4348-8fb1-5d04dad9bfcb
2,How have the advancements in GPT-4o and its pr...,"The advancements in GPT-4o, which is significa...",None,"The advancements in GPT-4o, particularly its p...",1,0,0,0.262459,43ad4982-4ca3-4522-aa0d-c3e82b7aaa32,2d7fad8c-d4ed-443b-9f8c-764a5d6ce2f1
3,How has Google's Gemini series contributed to ...,Google's Gemini series has significantly contr...,None,Google's Gemini series has significantly contr...,1,0,0,0.511032,a6ff2de5-84cb-4426-a68e-36d5303dac3f,6cd8e151-2fd5-41f3-b0b2-0738c4787fe3
4,How did the shift from universal access to sub...,The shift from universal access to subscriptio...,None,The shift from universal access to subscriptio...,1,1,0,0.268907,3e58700d-9104-47a9-a42f-167296fc4304,fcb44388-0720-423a-b327-16770993fe5f
5,How have advancements in GPT-4 and the competi...,Advancements in GPT-4 and the competition from...,None,Advancements in GPT-4 and the subsequent compe...,0,0,1,0.218723,69ee214c-8534-464b-b645-45fab132366b,d0f02ee4-9702-490e-a08a-45a10614d192
6,How has the evolution of prompt-driven app gen...,The evolution of prompt-driven app generation ...,None,"In 2024, prompt-driven app generation has beco...",1,0,0,0.321675,614d1d8d-c14c-420b-9263-61a8a13babb2,504accdb-37dc-4530-8cbd-00eb00b0a4fb
7,How have advancements in GPT-4 and the competi...,Advancements in GPT-4 and the competition from...,None,Advancements in GPT-4 and the subsequent compe...,1,0,1,0.199813,14ee9a64-f2ce-4371-ac08-0e9598e06475,9fa8e7da-14db-4d07-aec1-ecf7cfe2b8f1
8,How does Apple's MLX library enhance the perfo...,Apple's MLX library enhances the performance o...,None,"Apple's MLX library, described as an array fra...",1,0,0,0.162245,19cd73dd-141c-4815-b890-0f47bfdd303a,ee91d9b9-0b88-4fc8-9c57-96a10d6c2934
9,What are the key features and impacts of Claud...,Claude 3.5 Sonnet introduced a significant fea...,None,Claude 3.5 Sonnet introduced groundbreaking fe...,1,0,1,0.180664,eaea04ea-774a-4ef1-a08f-6d87be1db48d,60109b04-1e39-4f9c-8c6d-5533c5c09a1a


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [36]:
rag_documents = docs

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?
Larger chunkc sizes provide more context (more complete) within a segment but the larger size means fewer chunks can fit in the LLM's context window.  The larger chunk sizes might provide better context with the surrounding information but that can also mean it includes irrelevant information.  The larger chuncks also create fewer total vectors to search through but the vectors have greater context but you'll lose out on the granularity of matching specific details.  The larger chunk size will keep related concepts together and provide better context for the LLMs to generate answers and it needs to be manged to fit multiple chunks in context.

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?
 Using a larger model can better distinguish between subtle differences in meaning due to the larger dimensionality.  The larger model should be more accurate as it has been trained on more data and should pick up on more complex linguistic patterns.  The better embeddings lead to more accurate similarity searches in the vector databas and reduce hallucinations.

In [39]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [40]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [41]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [42]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Agents, my friend, are these mysterious AI systems that are supposed to roll up their sleeves and act on your behalf, kind of like a digital assistant or a travel agent. But here\'s the kicker—they\'re frustratingly vague and not clearly defined, leading to a whole lot of confusion. Some folks think of them as LLMs (Large Language Models) with tools, while others have a more traditional view of agents as things that make choices for you. The buzz around them is real, but actual examples in production? That\'s like chasing a phantom—still waiting on that! So, in a nutshell, agents are those elusive AI helpers we\'re all curious about, but they’re still in the "coming soon" phase, plagued by challenges like gullibility and a lack of clarity.'

Finally, we can evaluate the new chain on the same test set!

In [43]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'perfect-click-58' at:
https://smith.langchain.com/o/2928bae9-0c08-4543-b4d4-5b2f0a267e9d/datasets/00704a1a-6702-4a89-8b26-920db81b70ae/compare?selectedSessions=cba5fe52-2f38-42e5-903b-b3198157a281




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How did the developments in GPT-4 and Claude A...,"Yo, let me break it down for you! The developm...",None,"In 2023, GPT-4 and Claude Artifacts significan...",0,0,1,5.127867,c59f1bae-1f08-4850-a9ef-0c70a42536f4,abed975e-8b48-46b8-ac25-34a55c03bbc8
1,How did the developments in Large Language Mod...,I don't know.,None,"In 2023, Large Language Models (LLMs) marked a...",0,0,0,0.983946,e2093edc-a5c9-4077-ae52-81962af78e16,2996d568-66f5-4f25-af14-03666cbfd7bc
2,How have the advancements in GPT-4o and its pr...,"Yo, check it out! With the rollout of GPT-4o, ...",None,"The advancements in GPT-4o, particularly its p...",1,1,1,4.887934,43ad4982-4ca3-4522-aa0d-c3e82b7aaa32,7ba8d7bc-81ae-48d3-9228-ad2a106150dd
3,How has Google's Gemini series contributed to ...,"Yo, Google's Gemini series has totally flipped...",None,Google's Gemini series has significantly contr...,1,1,1,3.330325,a6ff2de5-84cb-4426-a68e-36d5303dac3f,e5711b98-0b86-47d1-96e1-83623b61f175
4,How did the shift from universal access to sub...,The shift from universal access to subscriptio...,None,The shift from universal access to subscriptio...,1,1,1,4.200185,3e58700d-9104-47a9-a42f-167296fc4304,3a18818b-7c2b-417e-a9c1-712527e6b223
5,How have advancements in GPT-4 and the competi...,"Yo, the landscape of large language models (LL...",None,Advancements in GPT-4 and the subsequent compe...,1,0,1,3.781439,69ee214c-8534-464b-b645-45fab132366b,e88c0b79-de2f-47e8-8a42-deaff65ce4f0
6,How has the evolution of prompt-driven app gen...,"Yo, the evolution of prompt-driven app generat...",None,"In 2024, prompt-driven app generation has beco...",1,1,1,4.236311,614d1d8d-c14c-420b-9263-61a8a13babb2,c4409d16-582b-40eb-a9cb-b61e9d6eeb3c
7,How have advancements in GPT-4 and the competi...,"Yo, the landscape of large language models (LL...",None,Advancements in GPT-4 and the subsequent compe...,1,0,1,0.525805,14ee9a64-f2ce-4371-ac08-0e9598e06475,f1d90f34-d734-409b-9fbe-d946a9ca67bc
8,How does Apple's MLX library enhance the perfo...,Apple's MLX library is a game changer for runn...,None,"Apple's MLX library, described as an array fra...",1,1,1,3.330101,19cd73dd-141c-4815-b890-0f47bfdd303a,a8c99225-c075-457d-8d7f-0b950dde4a3d
9,What are the key features and impacts of Claud...,"Yo, Claude 3.5 Sonnet is a total game changer ...",None,Claude 3.5 Sonnet introduced groundbreaking fe...,1,1,1,5.897723,eaea04ea-774a-4ef1-a08f-6d87be1db48d,6d724526-4097-4457-9f3f-f2aed9e77bfa


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.